# IMPORT MODULES

In [1]:
import pandas as pd

# READ CSV


How many views (i.e., rows of the table) came from each utm_source?

In [8]:
ad_clicks = pd.read_csv("ad_clicks.csv")
ad_clicks.head(10)

,user_id,utm_source,day,ad_click_timestamp,experimental_group
0,008b7c6c-7272-471e-b90e-930d548bd8d7,google,6 - Saturday,7:18,A
1,009abb94-5e14-4b6c-bb1c-4f4df7aa7557,facebook,7 - Sunday,NaN,B
2,00f5d532-ed58-4570-b6d2-768df5f41aed,twitter,2 - Tuesday,NaN,A
3,011adc64-0f44-4fd9-a0bb-f1506d2ad439,google,2 - Tuesday,NaN,B
4,012137e6-7ae7-4649-af68-205b4702169c,facebook,7 - Sunday,NaN,B
5,013b0072-7b72-40e7-b698-98b4d0c9967f,facebook,1 - Monday,NaN,A
6,0153d85b-7660-4c39-92eb-1e1acd023280,google,4 - Thursday,NaN,A
7,01555297-d6e6-49ae-aeba-1b196fdbb09f,google,3 - Wednesday,NaN,A
8,018cea61-19ea-4119-895b-1a4309ccb148,email,1 - Monday,18:33,A
9,01a210c3-fde0-4e6f-8efd-4f0e38730ae6,email,2 - Tuesday,15:21,B


# Prepare THE DATA

In [7]:
ad_clicks = pd.read_csv("ad_clicks.csv")
print(ad_clicks.groupby('utm_source').user_id.count().reset_index())

  utm_source  user_id
0      email      255
1   facebook      504
2     google      680
3    twitter      215


Create a new column called is_click, which is True if ad_click_timestamp is not null and False otherwise.


In [10]:
ad_clicks['is_click'] = ~ad_clicks.ad_click_timestamp.isnull()
ad_clicks.head(10)

,user_id,utm_source,day,ad_click_timestamp,experimental_group,is_click
0,008b7c6c-7272-471e-b90e-930d548bd8d7,google,6 - Saturday,7:18,A,True
1,009abb94-5e14-4b6c-bb1c-4f4df7aa7557,facebook,7 - Sunday,NaN,B,False
2,00f5d532-ed58-4570-b6d2-768df5f41aed,twitter,2 - Tuesday,NaN,A,False
3,011adc64-0f44-4fd9-a0bb-f1506d2ad439,google,2 - Tuesday,NaN,B,False
4,012137e6-7ae7-4649-af68-205b4702169c,facebook,7 - Sunday,NaN,B,False
5,013b0072-7b72-40e7-b698-98b4d0c9967f,facebook,1 - Monday,NaN,A,False
6,0153d85b-7660-4c39-92eb-1e1acd023280,google,4 - Thursday,NaN,A,False
7,01555297-d6e6-49ae-aeba-1b196fdbb09f,google,3 - Wednesday,NaN,A,False
8,018cea61-19ea-4119-895b-1a4309ccb148,email,1 - Monday,18:33,A,True
9,01a210c3-fde0-4e6f-8efd-4f0e38730ae6,email,2 - Tuesday,15:21,B,True


# Grouping by utm_source & is_click and counting the number

In [12]:
clicks_by_source= ad_clicks.groupby(['utm_source','is_click']).user_id.count().reset_index()
clicks_by_source.head(10)

,utm_source,is_click,user_id
0,email,False,175
1,email,True,80
2,facebook,False,324
3,facebook,True,180
4,google,False,441
5,google,True,239
6,twitter,False,149
7,twitter,True,66


# Pivot the data 

The columns are is_click

In [14]:
clicks_by_source_pivot = clicks_by_source\
    .pivot(index ='utm_source',
       columns='is_click',
       values ='user_id').reset_index()  
clicks_by_source_pivot.head(10)

is_click,utm_source,False,True
0,email,175,80
1,facebook,324,180
2,google,441,239
3,twitter,149,66


# Percent of clicking


In [17]:
clicks_by_source_pivot['percent_clicked ']= round(((clicks_by_source_pivot[True]/(clicks_by_source_pivot[True] + clicks_by_source_pivot[False])))*100,2)
clicks_by_source_pivot.head()

is_click,utm_source,False,True,percent_clicked
0,email,175,80,31.37
1,facebook,324,180,35.71
2,google,441,239,35.15
3,twitter,149,66,30.70


# Experimental_group

Group by experimental_group and count the number of users

And check to see if a greater percentage of users clicked on Ad A or Ad B.

In [19]:
experimental_group_numbers = ad_clicks.groupby(['experimental_group','is_click']).user_id.count().reset_index()
experimental_group_numbers_pivot = experimental_group_numbers.pivot\
(index= 'experimental_group',
 columns ='is_click',
 values='user_id').reset_index()
experimental_group_numbers_pivot.head()

is_click,experimental_group,False,True
0,A,517,310
1,B,572,255


Creating two DataFrames: a_clicks and b_clicks, which contain only the results for A group and B group

In [20]:
a_clicks = ad_clicks[ad_clicks.experimental_group == 'A']
b_clicks = ad_clicks[ad_clicks.experimental_group == 'B']

Calculate the percent of users who clicked on the ad by day

In [21]:
is_click_by_days= ad_clicks.groupby(['is_click','day']).user_id.count().reset_index()
is_click_by_days_pivot = is_click_by_days.pivot\
(index ='day',
 columns ='is_click',
 values= 'user_id').reset_index()
is_click_by_days_pivot['percentage'] = is_click_by_days_pivot[True]/(is_click_by_days_pivot[True] + is_click_by_days_pivot[False] )
is_click_by_days_pivot.head()

is_click,day,False,True,percentage
0,1 - Monday,151,75,0.331858
1,2 - Tuesday,150,88,0.369748
2,3 - Wednesday,175,73,0.294355
3,4 - Thursday,156,76,0.327586
4,5 - Friday,167,89,0.347656
